# 欠損データ補完スクリプト (Colab用)

このノートブックは、`database.csv` や `database_nar.csv` の空のカラム（主に過去のレース成績や血統情報）をnetkeibaからスクレイピングして埋めるためのものです。

## 手順
1. 左側のファイルアイコンから、補完したい `database.csv` (または `_nar.csv`) をアップロードしてください。
2. 以下のセルを順番に実行してください。
3. 完了すると、補完されたファイル (例: `filled_database.csv`) が生成されるので、ダウンロードして元のファイルと差し替えてください。

In [ ]:
!pip install pandas requests beautifulsoup4 tqdm

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import re

# === 設定 ===
TARGET_FILE = "database_nar.csv" # 補完したいファイル名 (適宜変更してください)
OUTPUT_FILE = "filled_database_nar.csv"

# スクレイピングの間隔 (秒) - サーバー負荷軽減のため必ず1秒以上空ける
INTERVAL = 1.0 

def get_soup(url):
    try:
        r = requests.get(url)
        r.encoding = r.apparent_encoding
        time.sleep(INTERVAL)
        return BeautifulSoup(r.text, 'html.parser')
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def scrape_horse_detail(horse_id):
    """
    馬のプロフィールページから血統と過去レース情報を取得
    """
    url = f"https://db.netkeiba.com/horse/{horse_id}"
    soup = get_soup(url)
    if not soup:
        return None

    data = {}

    # 1. 血統 (Pedigree)
    # father: .blood_table tr:nth-child(1) td:nth-child(1)
    try:
        blood_table = soup.select_one(".blood_table")
        if blood_table:
            # 父
            father_tag = blood_table.select_one("tr:nth-child(1) td:nth-child(1) a")
            if father_tag: data['father'] = father_tag.text.strip()
            
            # 母
            mother_tag = blood_table.select_one("tr:nth-child(3) td:nth-child(1) a")
            if mother_tag: data['mother'] = mother_tag.text.strip()
            
            # 母父 (BMS) - 母の父
            bms_tag = blood_table.select_one("tr:nth-child(3) td:nth-child(2) a")
            if bms_tag: data['bms'] = bms_tag.text.strip()
    except Exception as e:
        print(f"Pedigree error for {horse_id}: {e}")

    # 2. 過去レース (Past Races)
    # table class="db_h_race_results"
    # 最新5走を取得
    try:
        hist_table = soup.select_one("table.db_h_race_results")
        if hist_table:
            rows = hist_table.select("tbody tr")
            past_idx = 1
            # 時系列降順になっているはず
            for row in rows:
                if past_idx > 5: break
                
                # クラスによるフィルタリング等は一旦せず、単純に最近のものを取る
                cols = row.select("td")
                if len(cols) < 15: continue

                # 日付, 開催, ... 着順 ...
                # 0:日付, 1:開催, 2:天気, 3:R, 4:レース名, ... 11:着順
                try:
                    date_str = cols[0].text.strip()
                    rank = cols[11].text.strip()
                    
                    # rankが数字でない(取消など)場合はスキップするか、そのまま入れるか。
                    # 学習に使うなら数字が望ましいが、一旦そのまま。
                    
                    prefix = f"past_{past_idx}"
                    data[f"{prefix}_date"] = date_str
                    data[f"{prefix}_rank"] = rank
                    data[f"{prefix}_race_name"] = cols[4].text.strip()
                    
                    # 距離・馬場 (芝1600 etc)
                    dist_raw = cols[14].text.strip() 
                    # e.g., 芝1600 or ダ1200
                    if "芝" in dist_raw:
                       data[f"{prefix}_course_type"] = "芝"
                       data[f"{prefix}_distance"] = getattr(re.search(r'\d+', dist_raw), 'group', lambda: '')()
                    elif "ダ" in dist_raw:
                       data[f"{prefix}_course_type"] = "ダ"
                       data[f"{prefix}_distance"] = getattr(re.search(r'\d+', dist_raw), 'group', lambda: '')()
                    
                    data[f"{prefix}_time"] = cols[17].text.strip()
                    
                    # 通過 (Run Style hint)
                    pas = cols[20].text.strip()
                    # 4角位置を簡易的にrun_styleとする (1なら逃げ、など)
                    # 詳しくはfeatures.pyと合わせる必要があるが、簡易的に保存
                    
                    data[f"{prefix}_last_3f"] = cols[22].text.strip()
                    data[f"{prefix}_horse_weight"] = cols[23].text.strip()

                    past_idx += 1
                except Exception as ex:
                    print(f"Row parse error: {ex}")
                    continue
    except Exception as e:
        print(f"History error for {horse_id}: {e}")

    return data



In [ ]:
# メイン処理
try:
    df = pd.read_csv(TARGET_FILE)
    print(f"Loaded {len(df)} rows.")

    # 必要なカラムが存在しない場合は作成
    cols_to_ensure = ['father', 'mother', 'bms']
    for i in range(1, 6):
        cols_to_ensure += [
            f'past_{i}_date', f'past_{i}_rank', f'past_{i}_race_name', 
            f'past_{i}_distance', f'past_{i}_course_type', f'past_{i}_time', 
            f'past_{i}_last_3f', f'past_{i}_horse_weight'
        ]
    
    for c in cols_to_ensure:
        if c not in df.columns:
            df[c] = None

    # 進捗表示用
    total_updated = 0
    
    # horse_idごとにグループ化して処理（同じ馬を何度もスクレイピングしないため）
    # しかし、行ごとに異なる時点のデータを求めているわけではなく、馬自体の固定情報(血統)と最新履歴を取るならば
    # 重複除去したhorse_idリストを作って辞書にするのが効率的
    unique_horse_ids = df['horse_id'].dropna().unique()
    print(f"Unique horses to check: {len(unique_horse_ids)}")

    # 既にデータが埋まっているかチェックするロジックを入れるとさらに良いが、
    # ここでは簡易に「まだ辞書にない馬」を取得する
    horse_data_cache = {}

    for hid in tqdm(unique_horse_ids):
        # int float変換ケア
        try:
            hid_str = str(int(float(hid)))
        except:
            hid_str = str(hid)
        
        # Check if we assume fetching is needed (e.g., check first row of this horse in df)
        # 全件チェックは重いので、無条件に取得して上書きするスタイルにする（確実性重視）
        
        scraped_data = scrape_horse_detail(hid_str)
        
        if scraped_data:
            horse_data_cache[hid] = scraped_data
            total_updated += 1
        
        # Colabの制限を避けるため適宜保存などを推奨
        if total_updated % 100 == 0:
            print(f"Processed {total_updated} horses...")

    print("Applying data to dataframe...")
    
    # DataFrameに反映
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        hid = row['horse_id']
        if pd.isna(hid) or hid not in horse_data_cache:
            continue
            
        h_data = horse_data_cache[hid]
        # 血統は常に上書き
        if 'father' in h_data: df.at[idx, 'father'] = h_data['father']
        if 'mother' in h_data: df.at[idx, 'mother'] = h_data['mother']
        if 'bms' in h_data: df.at[idx, 'bms'] = h_data['bms']
        
        # 過去データ
        # 注意: database.csvの各行は「ある時点のレース」なので、
        # 本来はそのレース日付より過去の戦績だけを入れるべきです（リーク防止）。
        # しかし、この簡易スクリプトでは「最新5走」を取ってきて埋めるため、
        # 未来のデータが入る可能性があります。
        # **厳密な学習**を行う場合は、ここを「日付比較」するロジックにする必要があります。
        
        current_date = pd.to_datetime(row['日付'])
        
        # h_dataの中身 (past_1_date, past_1_rank...) は「最新順」になっている
        # これを current_date より古いものだけフィルタして再割り当てする
        
        valid_past_races = []
        # Check up to 10 past races scraped (if we scraped more) - currenly scraped 5
        # Scraper function above scrapes top 5. 
        # Let's verify dates.
        
        # 簡易実装: 日付チェック付きで埋める
        # 一旦 scraped_data からリスト形式でレースを復元
        temp_races = []
        for i in range(1, 6):
            d_key = f"past_{i}_date"
            if d_key in h_data and h_data[d_key]:
                r = {k.replace(f"past_{i}_", ""): v for k, v in h_data.items() if k.startswith(f"past_{i}_")}
                temp_races.append(r)
        
        # 日付比較
        # date format in netkeiba usually YYYY/MM/DD, df usually YYYY-MM-DD or YYYY年...
        filled_count = 0
        for r in temp_races:
            try:
                # parse race date
                p_date = pd.to_datetime(r['date'])
                if p_date < current_date:
                    filled_count += 1
                    p_idx = filled_count
                    if p_idx > 5: break
                    
                    # 埋める
                    for k, v in r.items():
                        col_name = f"past_{p_idx}_{k}"
                        if col_name in df.columns:
                            df.at[idx, col_name] = v
            except:
                continue

    # 保存
    df.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved to {OUTPUT_FILE}")

except Exception as e:
    print(f"Fatal error: {e}")